# HDFS Knowledge Graph Visualization

This notebook demonstrates the use of the HDFS Knowledge Graph module to build, analyze, and visualize knowledge graphs from HDFS log data.

In [1]:
import sys
import os
sys.path.append('../src')  # Add src directory to path
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from IPython.display import display, HTML

# Import functions from the graph module
from graph import (
    create_hdfs_knowledge_graph, 
    find_common_patterns,
    analyze_graph,
    visualize_graph,
    save_knowledge_graph,
    load_knowledge_graph,
    get_trace_subgraph
)

ModuleNotFoundError: No module named 'matplotlib'

## 1. Creating a Knowledge Graph

First, let's create a knowledge graph from the HDFS log data.

In [ ]:
# Specify the data directory - adjust path as needed
data_dir = "../data/processed"

# Create the knowledge graph
kg = create_hdfs_knowledge_graph(data_dir)

# Alternatively, if you've already saved a graph, you can load it:
# kg = load_knowledge_graph("../reports/hdfs_knowledge_graph.pkl", data_dir)

## 2. Analyzing Graph Statistics

Let's examine the statistics of our knowledge graph to better understand its structure.

In [ ]:
# Get graph statistics
stats = analyze_graph(kg)

# Display basic graph properties
print(f"Total nodes: {stats['num_nodes']}")
print(f"Total edges: {stats['num_edges']}")
print(f"Number of connected components: {stats['num_components']}")

# Display node type distribution
print("\nNode types:")
for node_type, count in stats['node_types'].items():
    print(f"  {node_type}: {count}")

# Display edge relation distribution
print("\nEdge relations:")
for relation, count in stats['edge_relations'].items():
    print(f"  {relation}: {count}")

## 3. Identifying Common Patterns

Let's find common event sequences in the HDFS logs.

In [ ]:
# Identify common event patterns
common_sequences = find_common_patterns(kg)

# Display the top sequences in a more readable format
print("\nTop common event sequences:")
for i, (seq, count) in enumerate(common_sequences[:5], 1):
    print(f"\nSequence {i} (occurs {count} times):")
    for j, event_id in enumerate(seq, 1):
        print(f"  Step {j}: Event {event_id}")

## 4. Visualizing the Graph

Now, let's visualize the knowledge graph to better understand its structure.

In [ ]:
# Visualize the graph with a limited number of nodes for clarity
# This displays the graph in the notebook
visualize_graph(kg, max_nodes=100)

In [ ]:
# Save a larger visualization to a file
os.makedirs("../reports", exist_ok=True)
output_path = "../reports/hdfs_knowledge_graph_large.png"
visualize_graph(kg, max_nodes=300, output_path=output_path)

# Display the saved image
from IPython.display import Image
Image(filename=output_path, width=800)

## 5. Examining a Specific Trace

Let's extract and visualize a specific trace for deeper analysis.

In [ ]:
# Get a list of block IDs in the knowledge graph
block_ids = [d['block_id'] for _, d in kg.graph.nodes(data=True) 
             if d.get('type') == 'Trace' and 'block_id' in d]

# Display some sample block IDs
print(f"Found {len(block_ids)} block IDs in the graph.")
print("Sample block IDs:")
for block_id in block_ids[:5]:
    print(f"  {block_id}")

In [ ]:
# Select a block ID to examine (using the first one from our sample)
if block_ids:
    selected_block_id = block_ids[0]
    
    # Get the subgraph for this trace
    trace_subgraph = get_trace_subgraph(kg, selected_block_id)
    
    if trace_subgraph:
        print(f"Extracted subgraph for block {selected_block_id} with {trace_subgraph.number_of_nodes()} nodes")
        
        # Visualize the trace subgraph
        plt.figure(figsize=(12, 8))
        pos = nx.spring_layout(trace_subgraph, seed=42)
        
        # Create node colors based on node type
        node_colors = []
        for node in trace_subgraph.nodes():
            node_type = trace_subgraph.nodes[node].get('type', '')
            if node_type == 'Component':
                node_colors.append('skyblue')
            elif node_type == 'EventType':
                node_colors.append('lightgreen')
            elif node_type == 'Event':
                node_colors.append('orange')
            elif node_type == 'Trace':
                node_colors.append('green')
            else:
                node_colors.append('gray')
        
        nx.draw(trace_subgraph, pos, with_labels=True, node_color=node_colors,
                font_size=8, node_size=500, font_weight='bold',
                edge_color='gray', width=0.5, alpha=0.8)
        
        plt.title(f"Trace Subgraph for Block {selected_block_id}")
        plt.show()
    else:
        print(f"No subgraph found for block {selected_block_id}")
else:
    print("No block IDs found in the graph.")

## 6. Saving the Knowledge Graph

Let's save our knowledge graph for future use.

In [ ]:
# Create the reports directory if it doesn't exist
os.makedirs("../reports", exist_ok=True)

# Save the graph
save_knowledge_graph(kg, "../reports/hdfs_knowledge_graph.pkl")

## 7. Advanced Analysis: Event Type Distribution

Let's analyze the distribution of event types in our logs.

In [ ]:
# Extract event type nodes
event_type_nodes = [n for n, d in kg.graph.nodes(data=True) if d.get('type') == 'EventType']

# Count event instances per event type
event_type_counts = {}
for node in event_type_nodes:
    event_id = kg.graph.nodes[node].get('event_id')
    template = kg.graph.nodes[node].get('template', 'Unknown')
    # Count instances pointing to this event type
    instances = sum(1 for _, target, data in kg.graph.edges(data=True) 
                    if target == node and data.get('relation') == 'instance_of')
    event_type_counts[event_id] = (instances, template)

# Sort by frequency
sorted_event_types = sorted(event_type_counts.items(), key=lambda x: x[1][0], reverse=True)

# Display the most common event types
print("Most common event types:")
for event_id, (count, template) in sorted_event_types[:10]:
    print(f"EventID: {event_id}, Occurrences: {count}")
    print(f"Template: {template}\n")

In [ ]:
# Visualize event type distribution
top_events = sorted_event_types[:15]  # Top 15 event types
event_ids = [event_id for event_id, _ in top_events]
counts = [count for _, (count, _) in top_events]

plt.figure(figsize=(14, 8))
bars = plt.bar(event_ids, counts, color='skyblue')
plt.xlabel('Event ID')
plt.ylabel('Number of Occurrences')
plt.title('Distribution of Top 15 Event Types')
plt.xticks(rotation=45)
plt.tight_layout()

# Add count labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
            f'{height:.0f}', ha='center', va='bottom')

plt.show()

## 8. Component Analysis

Let's analyze the distribution and relationship between components in the system.

In [ ]:
# Extract component nodes
component_nodes = [n for n, d in kg.graph.nodes(data=True) if d.get('type') == 'Component']
component_names = [kg.graph.nodes[n].get('name', 'Unknown') for n in component_nodes]

print(f"Found {len(component_nodes)} distinct components:")
for name in component_names:
    print(f"  - {name}")

# Count events by component
component_event_counts = {}
for component_node in component_nodes:
    component_name = kg.graph.nodes[component_node].get('name', 'Unknown')
    # Count events connected to this component
    events = sum(1 for source, target, data in kg.graph.edges(data=True) 
                if target == component_node and data.get('relation') == 'executed_by')
    component_event_counts[component_name] = events

# Display event counts by component
print("\nEvent counts by component:")
for component, count in sorted(component_event_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {component}: {count} events")

# Visualize component distribution
plt.figure(figsize=(10, 6))
plt.pie([count for count in component_event_counts.values()], 
        labels=[f"{comp} ({count})" for comp, count in component_event_counts.items()],
        autopct='%1.1f%%',
        startangle=90,
        shadow=True)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title('Distribution of Events by Component')
plt.tight_layout()
plt.show()

## 9. Analyzing Temporal Patterns

Let's look at the temporal distribution of events.

In [ ]:
# Extract event nodes with timestamps
event_nodes = []
for node, data in kg.graph.nodes(data=True):
    if data.get('type') == 'Event' and 'timestamp' in data:
        event_nodes.append((node, data['timestamp']))

# Convert timestamps to pandas datetime
if event_nodes:
    import pandas as pd
    from datetime import datetime
    
    # Attempt to parse timestamps
    timestamps = []
    for _, ts_str in event_nodes:
        try:
            ts = pd.to_datetime(ts_str)
            timestamps.append(ts)
        except:
            # Skip unparseable timestamps
            continue
    
    if timestamps:
        # Create a time series of event counts
        ts_series = pd.Series(timestamps).value_counts().sort_index()
        
        # Resample to minute frequency to see patterns
        minute_counts = ts_series.resample('1min').sum().fillna(0)
        
        # Plot time series
        plt.figure(figsize=(15, 6))
        minute_counts.plot()
        plt.title('Events Over Time (1-minute intervals)')
        plt.xlabel('Time')
        plt.ylabel('Number of Events')
        plt.grid(True)
        plt.show()
        
        # Show more detailed statistics
        print(f"Temporal analysis of {len(timestamps)} events:")
        print(f"First event: {timestamps[0]}")
        print(f"Last event: {timestamps[-1]}")
        print(f"Total time span: {timestamps[-1] - timestamps[0]}")
        print(f"Average events per minute: {len(timestamps) / ((timestamps[-1] - timestamps[0]).total_seconds() / 60):.2f}")
    else:
        print("No valid timestamps found in event data.")
else:
    print("No event nodes with timestamps found in the graph.")

## 10. Conclusion and Next Steps

In this notebook, we have:
1. Built a knowledge graph from HDFS log data
2. Analyzed the graph structure and statistics
3. Identified common event patterns
4. Visualized the graph and specific traces
5. Performed more detailed analyses of event types, components, and temporal patterns

Potential next steps could include:
- Anomaly detection based on unusual event patterns
- Predictive modeling for system failures
- Root cause analysis using graph traversal algorithms
- Integration with real-time monitoring systems